### Demonstrates use of the build_pairpro function for converting learn2thermDB files
*To do: Update this once OMA is onboarded*  
*Note: Sequence alignment is not calculated since it is supplied in learn2thermDB*

In [1]:
import os

from pairpro import preprocessing

In [19]:
# Connect to example database file
con, tables = preprocessing.connect_db('../data/l2t_50k.db')

Connecting to database...
Connection established! Execution time: 0.02022552490234375 seconds


In [20]:
# This is the general structure of a learn2thermDB file
print(tables)

       table_name
0  taxa_pairs_lab
1      taxa_pairs
2            taxa
3        proteins
4   protein_pairs


In [21]:
# This is a temporary directory for storing the output database of the make_blast_df function
if not os.path.exists('tmp'):
    os.mkdir('tmp')

In [22]:
# Build example PairProphet database from example l2t

con, filename = preprocessing.build_pairpro(con, '/tmp/example_pairpro.db')

Constructing pairpro_taxa_pairs...
Finished constructing pairpro_taxa_pairs. Execution time:
          0.0023832321166992188 seconds
Constructing pairpro_taxa...
Finished constructing pairpro_taxa. Execution time:
          0.0021681785583496094 seconds
Filtering on ogt and 16S sequence parameters...
Finished filtering. Execution time: 0.0027234554290771484 seconds
Constructing pairpro_protein_pairs...
Finished constructing pairpro_protein_pairs. Execution time:
          0.014923572540283203 seconds
Constructing pairpro_proteins...
Finished constructing pairpro_proteins. Execution time:
          0.018204927444458008 seconds
Constructing final dataset...
Transferring data to new database /tmp/example_pairpro_4.db
Finishing up...
Connecting to database...
Connection established! Execution time: 0.018743515014648438 seconds
Finished. Total execution time: 0.1580944061279297 seconds


In [23]:
# Name of new database. The original learn2thermDB file should be unaffected by build_pairpro()
print(filename)

example_pairpro_4


In [27]:
'''
This is the structure of the PairProphet data table saved in the new db file
Note that the con object returned by build_pairpro() is not the same as the input con
which is closed during execution. This protects the original db file.
'''
con.execute("""SELECT * FROM pairpro.final LIMIT 5""").df()

,thermo_pid,meso_pid,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,local_E_value,query_align_start,query_align_end,subject_align_end,subject_align_start,...,thermo_taxid,meso_taxid,m_ogt,t_ogt,ogt_difference,m_protein_seq,t_protein_seq,meso_pdb,thermo_pdb,pair_id
0,A0A2W7RQ16,A0A327VJ32,0.303279,0.291339,0.290196,2.170000e-12,7,127,128,7,...,1004304,1539050,25.0,45.0,20.0,MTRINYSSGATWEDKVGYSRAVRTGNIIEVSGTVAEDEGKVVAEGN...,MHYQRISSGSVYEEQMCYSRAVVAAPFIFVSGTTGFNYATMTIADD...,NaN,NaN,6767
1,A0A2W7RTL2,A0A327VMA9,0.644068,0.567164,0.584615,4.180000e-24,6,64,61,3,...,1004304,1539050,25.0,45.0,20.0,MQTGVVKFFNETKGFGFIKIEGTNQEIFVHVSGIKESIGENDRVVF...,MSTKITGTVKFFNEEKGFGFIKHDDSNKETFVHANGLIDQIEANDK...,NaN,NaN,6880
2,A0A2W7RYG8,A0A327W1Z1,0.538462,0.477273,0.473684,4.340000e-40,12,128,130,14,...,1004304,1539050,25.0,45.0,20.0,MARKGNKNKRRNLEKLSHKYRLVIMNDETYDEVTSFKLSRMSVYIA...,MAKQIFETNTFKRLRNQYRLVIINDDTYEELVTFKLSRLSVYVAFS...,NaN,NaN,7492
3,A0A2W7RR29,A0A327W5V0,0.660714,0.599190,0.609053,7.720000e-113,24,247,239,16,...,1004304,1539050,25.0,45.0,20.0,MMSSMSKNINTHLVEIKDYFKVAISVDCVIFGFNNDELKVLLIESD...,MKPKSVQTEPAAQRPLITDVKALVNSYPRVPITVDCVIFGFDGEEL...,NaN,NaN,7728
4,A0A2W7RR29,A0A327VM41,0.414747,0.364372,0.376569,4.440000e-58,28,243,227,11,...,1004304,1539050,25.0,45.0,20.0,MNAASFYSKAPRHLVAVDCIIFGFDEGKLKLLVIKRKVAPMAGEWS...,MKPKSVQTEPAAQRPLITDVKALVNSYPRVPITVDCVIFGFDGEEL...,NaN,NaN,6868


**Execute cell below to clean up tmp and database files**

In [29]:
con.close()
for f in os.listdir('tmp'):
    os.remove(os.path.join('tmp', f))
os.rmdir('tmp')